In [46]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!pip install janome

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-l

     |████████████████████████████████| 19.7MB 56.2MB/s 


In [2]:
from google.colab import drive 
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [21]:
#データの読み込み
import pandas as pd
df = pd.read_excel( "drive/My Drive/machine_learning/NLP/kana_nishino/kana_nishino.xlsx")
df.head()

,タイトル（／複数A面）,最高位,登場週数,累計売上(枚),発売日,歌詞
0,I,155,1,481,08.2.20,Darling I am here now そばにいさせて最終バス気にならないくらい寝ても醒...
1,glowly days,126,2,1044,08.4.23,いま風に抱かれて キミのもとへ吹いていけたらいいのにね名前をつぶやいた 聞えるかな？新しい部...
2,Style.,57,3,2787,08.8.13,はじまるよ キミとボクをつなぐ扉ノックすればどんな未来が僕らを待っているの?大人がそう求める...
3,MAKE UP,119,1,698,09.1.28,One day' Did the alarm clock go off?Wake up!まだ...
4,遠くても feat．WISE,40,12,9536,09.3.18,君と出会えたことがかけがえのない宝物今は会えなくてもAlways Love You『今何して...


In [22]:
df.isnull().sum()

タイトル（／複数A面）    0
最高位            0
登場週数           0
累計売上(枚)        0
発売日            0
歌詞             0
dtype: int64

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   タイトル（／複数A面）  34 non-null     object
 1   最高位          34 non-null     int64 
 2   登場週数         34 non-null     int64 
 3   累計売上(枚)      34 non-null     int64 
 4   発売日          34 non-null     object
 5   歌詞           34 non-null     object
dtypes: int64(3), object(3)
memory usage: 1.7+ KB


In [24]:
#歌詞の確認
df["歌詞"][0]

"Darling I am here now そばにいさせて最終バス気にならないくらい寝ても醒めても Do you always love me?ねぇ教えて1分1秒[I always] feel restless賭けて コケて 泣いて emotion 止まらない 何故？涙 brop in a 未来のooean Reaby to this hand 'Esoape' SOS!!I don't wanna know 埋め尽くしてよI don't wanna know キミのすべてI don't wanna know 代わりなんていないでしょ？I don't wanna know あの子の痕I don't wanna know 気になるんだI don't wanna know ずっとほどけないよう 離さないで 離れないでどれほどの濃い愛を味わって今私を抱きしめてくれたのだろう知れば知るほど 傷ついてゆく相性占い 人は弱い 先の見えないstrange引き devil 抜？'angel'保守 妄相Gate...fall引き angel 抜け! 'devil' 葛藤 Ready to this hand 'Esoape' SOS!!I don't wanna know 埋め尽くしてよI don't wanna know キミのすべてI don't wanna know 代わりなんていないでしょ？I don't wanna know あの子の影I don't wanna know 気になるんだI don't wanna know ここに繋ぐための過去なのに 過去だけど明日が嫌になったら イエスタディ 巻き戻せる？I don't wanna know どんな不安もI don't wanna know 見つめているよI don't wanna know 消えることはないでしょ？I don't wanna know すべての痕I don't wanna know 埋め尽くしてよI don't wanna know ずっとほどけないよう 離さないで 離れないでDarling' l am here now. l always love you"

In [30]:
#記号や英語の削除
import re
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\n', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', '？',"！","『","』",'「',"」","。","、","*",]

def clean_title(text):
    for punct in puncts:
        text = text.replace(punct, "")#記号の削除
    text = re.sub("[0-9a-zA-Z]+" , "" , text) #数字の削除、[0-9a-zA-Z]+これで英字も削除することができる。
    text = "".join(text)
    return text

print(clean_title(df["歌詞"][0]))

     そばにいさせて最終バス気にならないくらい寝ても醒めても     ねぇ教えて分秒   賭けて コケて 泣いて  止まらない 何故涙    未来の          埋め尽くしてよ    キミのすべて    代わりなんていないでしょ    あの子の痕    気になるんだ    ずっとほどけないよう 離さないで 離れないでどれほどの濃い愛を味わって今私を抱きしめてくれたのだろう知れば知るほど 傷ついてゆく相性占い 人は弱い 先の見えない引き  抜保守 妄相引き  抜け  葛藤          埋め尽くしてよ    キミのすべて    代わりなんていないでしょ    あの子の影    気になるんだ    ここに繋ぐための過去なのに 過去だけど明日が嫌になったら イエスタディ 巻き戻せる    どんな不安も    見つめているよ    消えることはないでしょ    すべての痕    埋め尽くしてよ    ずっとほどけないよう 離さないで 離れないで        


In [36]:
#歌詞から記号や英字を削除している。
df["clear_kashi"] = [clean_title(df["歌詞"][i]) for i in range(len(df["歌詞"]))]
df

,タイトル（／複数A面）,最高位,登場週数,累計売上(枚),発売日,歌詞,clear_kashi
0,I,155,1,481,08.2.20,Darling I am here now そばにいさせて最終バス気にならないくらい寝ても醒...,そばにいさせて最終バス気にならないくらい寝ても醒めても ねぇ教えて分秒 ...
1,glowly days,126,2,1044,08.4.23,いま風に抱かれて キミのもとへ吹いていけたらいいのにね名前をつぶやいた 聞えるかな？新しい部...,いま風に抱かれて キミのもとへ吹いていけたらいいのにね名前をつぶやいた 聞えるかな新しい部屋...
2,Style.,57,3,2787,08.8.13,はじまるよ キミとボクをつなぐ扉ノックすればどんな未来が僕らを待っているの?大人がそう求める...,はじまるよキミとボクをつなぐ扉ノックすればどんな未来が僕らを待っているの大人がそう求めるのは...
3,MAKE UP,119,1,698,09.1.28,One day' Did the alarm clock go off?Wake up!まだ...,まだまだ眠いの二度寝したいけど 慌ててメールなんで期待毎日日課だったおはよう...
4,遠くても feat．WISE,40,12,9536,09.3.18,君と出会えたことがかけがえのない宝物今は会えなくてもAlways Love You『今何して...,君と出会えたことがかけがえのない宝物今は会えなくても 今何してるの本当に友達なの不安になる...
5,君に会いたくなるから,19,8,12412,09.6.3,降りだした雨を見上げて君の声聞きたくなったきっと今どこかで君も同じ空を見てるの君とのメールも...,降りだした雨を見上げて君の声聞きたくなったきっと今どこかで君も同じ空を見てるの君とのメールも...
6,もっと…,6,31,65220,09.10.21,今すぐ会いたい もっと声が聞きたいこんなにも君だけ想ってるのに不安で仕方ない 何度も聞きたい...,今すぐ会いたい もっと声が聞きたいこんなにも君だけ想ってるのに不安で仕方ない 何度も聞きたい...
7,Dear…,7,23,52792,09.12.2,「じゃあね」って言ってからまだ5分もたってないのにすぐに会いたくてもう一度 oh babyギ...,「じゃあね」って言ってからまだ分もたってないのにすぐに会いたくてもう一度 ギュッとしてほし...
8,Best Friend,3,23,70883,10.2.24,ありがとう君がいてくれて本当よかったよどんな時だっていつも笑っていられる例えば、離れていても...,ありがとう君がいてくれて本当よかったよどんな時だっていつも笑っていられる例えば離れていても ...
9,会いたくて 会いたくて,2,25,97103,10.5.19,会いたくて 会いたくて 震える君想うほど遠く感じてもう一度聞かせて嘘でもあの日のように“好き...,会いたくて 会いたくて 震える君想うほど遠く感じてもう一度聞かせて嘘でもあの日のように好きだ...


ModuleNotFoundError: ignored

In [63]:
#MeCabで単語に分割している
import MeCab
m = MeCab.Tagger ('-Ochasen')

words=[]

for i in range(len(df["clear_kashi"])):
  node = m.parseToNode(df["clear_kashi"][i])
  while node:
      hinshi = node.feature.split(",")[0]
      if hinshi in ["名詞","動詞","形容詞"]:
          origin = node.feature.split(",")[6]
          words.append(origin)
      node = node.next

#単語の数カウント
import collections
c = collections.Counter(words)
print(c.most_common(20))

[('君', 192), ('いる', 155), ('する', 101), ('私', 90), ('の', 89), ('てる', 86), ('人', 78), ('好き', 65), ('なる', 59), ('*', 56), ('今', 55), ('何', 47), ('こと', 46), ('日', 44), ('くれる', 43), ('誰', 39), ('ある', 39), ('時', 38), ('心', 36), ('キミ', 35)]


In [69]:
#janomeで行ってみる。
from janome.tokenizer import Tokenizer

t= Tokenizer()
tokens = t.tokenize(df["clear_kashi"][0])
for token in tokens:
  partofspeech = token.part_of_speech.split(",")[0]#品詞の取得

名詞
助詞
動詞
動詞
助詞
名詞
名詞
名詞
助詞
動詞
助動詞
助詞
動詞
助詞
助詞
動詞
助詞
助詞
記号
感動詞
動詞
助詞
名詞
記号
動詞
助詞
記号
動詞
助詞
記号
動詞
助詞
記号
動詞
助動詞
記号
副詞
名詞
記号
名詞
助詞
記号
動詞
動詞
助詞
助詞
記号
名詞
助詞
名詞
記号
名詞
助詞
動詞
助動詞
助動詞
記号
連体詞
名詞
助詞
名詞
記号
名詞
助詞
動詞
名詞
助動詞
記号
副詞
動詞
助動詞
名詞
記号
動詞
助動詞
助詞
記号
動詞
助動詞
助詞
名詞
助詞
助詞
形容詞
名詞
助詞
動詞
助詞
名詞
名詞
助詞
動詞
助詞
動詞
助動詞
名詞
助動詞
助動詞
動詞
助詞
動詞
助詞
記号
動詞
助詞
動詞
名詞
名詞
記号
名詞
助詞
形容詞
記号
名詞
助詞
動詞
助動詞
名詞
記号
名詞
記号
名詞
助動詞
記号
動詞
記号
名詞
記号
動詞
動詞
助詞
助詞
記号
名詞
助詞
名詞
記号
名詞
助詞
動詞
助動詞
助動詞
記号
連体詞
名詞
助詞
名詞
記号
名詞
助詞
動詞
名詞
助動詞
記号
名詞
助詞
動詞
名詞
助詞
名詞
助動詞
助詞
記号
名詞
助動詞
助詞
名詞
助詞
名詞
助詞
動詞
助動詞
記号
名詞
記号
動詞
動詞
記号
連体詞
名詞
助詞
記号
動詞
助詞
動詞
助詞
記号
動詞
名詞
助詞
助動詞
助動詞
記号
名詞
助詞
名詞
記号
動詞
動詞
助詞
助詞
記号
副詞
動詞
助動詞
名詞
記号
動詞
助動詞
助詞
記号
動詞
助動詞
助詞


<generator object Tokenizer.__tokenize_stream at 0x7fc5f1e973b8>